In [ ]:
# Colab only: Run this cell to download/install packages
import sys
if 'google.colab' in sys.modules:
    !curl http://www.datasciencecourse.org/assignments/hw3_linear.tar.gz --output hw3_linear.tar.gz
    !tar -xzf hw3_linear.tar.gz
    !mv hw3_linear/* ./
    !pip install --upgrade --no-deps --force-reinstall git+https://github.com/locuslab/mugrade.git

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1230k  100 1230k    0     0  13.8M      0 --:--:-- --:--:-- --:--:-- 13.9M
  Cloning https://github.com/locuslab/mugrade.git to /tmp/pip-req-build-8l3dqkoz
  Running command git clone --filter=blob:none --quiet https://github.com/locuslab/mugrade.git /tmp/pip-req-build-8l3dqkoz
  Resolved https://github.com/locuslab/mugrade.git to commit b72e60d0e4de53204c7a49e1ca16ddb0f79237ce
  Preparing metadata (setup.py) ... done
  Created wheel for mugrade: filename=mugrade-1.1-py3-none-any.whl size=4120 sha256=9ad1fcd4df4614a63505d3c08eb6ddad33ead5db8dfffd540da92dc3058818eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lx9p842/wheels/41/c2/50/84457fdcb77bb99d9333659296b8b431528b294c909fd66c2b
Successfully built mugrade


# Linear Regression

In this homework, we are going to apply linear regression to the problem of predicting developer satisfaction based upon information about their careers, from a StackOverflow survey.  The data from this question is based on the [2019 StackOverflow Survey](https://insights.stackoverflow.com/survey/2019); accordingly, the subset bundled with this assignment is also released under the Open Database License (ODbL) v1.0.  For this problem, you should not use Scikit-Learn, but instead, implement all the least squares solutions manually.


### Q1 Data Parsing

The data from this survey has the values as described below.  Your eventual goal will be to predict (a numerical equivalent of) the `CarreerSat` column, indicating how satisfied the subject is with their career), based upon the other values.  Note that because the career satisfaction values are ordinal, we likely should not be predicting them with linear regression, but as an illustrative example, this is still a reasonable task.

The data set contains the following columns.

| Column | Sample | Does/is the respondent... | Type/Values |
| --- |:--- |:--- |:--- |
| **CareerSat** | 'vs' | satisfied with their career? | (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) -- corresponding to ({very, slightly}, {satisfied, dissatisfied}), neutral, and not applicable |
| MgrWant | 'n' | ...want to be a manager? | boolean |
| Age    | '22'   | age | integer     |
| CodeRevHrs | '2' | hours a week spent reviewing code | integer |
| ConvertedComp | '61000' | yearly compensation in 2019 USD | integer |
| Country | 'United States' | lives in country | string _(ignore in regression)_ |
| Dependents | 'n' | ...have children or other dependents. | boolean |
| DevEnvironVSC | 'y' | ...use Visual Studio Code | boolean |
| DevTypeFullStack | 'n' | ...identify as a full-stack developer | boolean |
| EdLevel | 'bachelors' | maximum education level | (`other`, `bachelors`, `masters`, `doctoral`) |
| EduOtherMOOC | 'y' | ...ever taken a Massively Open Online Course | boolean |
| EduOtherSelf | 'y' | ...ever taught themselves a new platform | boolean _(ignore in regression, this is  duplicate field)_ |
| Extraversion | 'y' | ...prefer in-person meetings to online meetings | boolean |
| GenderIsMan | 'y' | ...male | boolean |
| Hobbyist | 'n' | ...write code as a hobby? | boolean |
| MgrIdiot | 'very' | ...think their manager knows what they are doing? | (`NA`, `not`, `some`, `very`), in order of increasing confidence |
| OpSys | 'win' | which OS do they use? | (`win`, `mac`, `tux`, `NA`), for (Windows, Mac OSX, Linux-like, NA) |
| OpenSourcer | 'Never' | ...contribute to open-source projects? | (`never`, `year`, `month-year`, `month`), in increasing order of frequency |
| OrgSize | '100-499' | number of employees in organization? | (`NA`, `1`, `2-9`, `10-19`, `20-99`, `100-499`, `500-999`, `1,000-4,999`, `5,000-9,999`, `10,000+`) |
| Respondent | '4' | respondent ID from original data | integer _(ignore in regression)_ |
| Student | 'n' | ...currently a student? | boolean |
| UndergradMajorIsComputerScience | 'y' | ...majored in CS? | boolean |
| UnitTestsProcess | 'n' | ...use unit tests in their job? | boolean |
| WorkWeekHrs | '40' | hours a week worked | integer |
| YearsCode | 3 | years since first programming | integer |
| YearsCodePro | 0 | years programming professionally | integer |

When you load the data for linear regression, you will want to convert each type to a floating point value according to the columns above.  Specifically, for each column listed above, convert the value to a numeric quantity using the rules below.  Note that there are some fairly unnatural assumptions here (like converting NAs to 0.0), but these are largely to illustrate the methodology without adding too much additional processing work.

 - _boolean_ : `y`/`NA`/`n` assigned to `+1.0`/`0.0`/`0.0`
 - _integer_ : convert to `float`, preserving value. `NA` equals `0.0`.
 - _string_ : not included in regression; we'll use it later
 - CareerSat: Map (`vd`, `sd`, `ne`, `NA`, `ss`, `vs`) to (-2.0, -1.0, 0.0, 0.0, 1.0, 2.0)
 - EdLevel: Map (`other`, `bachelors`, `masters`, `doctoral`) to (0.0, 1.0, 1.5, 2.0)
 - MgrIdiot: Map (`NA`, `not`, `some`, `very`) to (-1.0, -1.0, 0.0, 1.0)
 - OpSys: Map (`win`, `mac`, `NA`, `tux`, `BSD`) to (-1.0, 0.0, 0.0, 1.0, 1.0)
 - OpenSourcer : Map (`never`, `year`, `month-year`, `month`) to (0.0, 0.5, 1.0, 2.0)
 - OrgSize: Map each range "$a$-$b$" to the value $ln(a)$. Treat `NA` as `ln(1.0) = 0`. We are converting an exponentially distributed range to a linearly distributed one.

Remove the columns listed above as being ignored.

Some hints:
  1. Load the csv file with `pd.read_csv(filname, dtype=str, keep_default_na=False)` to ensure that you load all columns as text (so you can do your own preprocessing), and ignore pandas's default conversion to NaN values.
  2. Use the `.apply()` function in pandas to convert

In [ ]:
import csv
import gzip
import math
import hashlib
import numpy as np
import pandas as pd
import mugrade

In [ ]:
# Conversion functions
def convert_boolean(value):
    if value == 'y':
        return 1.0
    else:
        return 0.0

def convert_integer(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0

def convert_CareerSat(value):
    mapping = {'vd': -2.0, 'sd': -1.0, 'ne': 0.0, 'NA': 0.0, 'ss': 1.0, 'vs': 2.0}
    return mapping.get(value, 0.0)

def convert_EdLevel(value):
    mapping = {'other': 0.0, 'bachelors': 1.0, 'masters': 1.5, 'doctoral': 2.0}
    return mapping.get(value, 0.0)

def convert_MgrIdiot(value):
    mapping = {'NA': -1.0, 'not': -1.0, 'some': 0.0, 'very': 1.0}
    return mapping.get(value, 0.0)

def convert_OpSys(value):
    mapping = {'win': -1.0, 'mac': 0.0, 'tux': 1.0, 'NA': 0.0}
    return mapping.get(value, 0.0)

def convert_OpenSourcer(value):
    mapping = {'never': 0.0, 'year': 0.5, 'month-year': 1.0, 'month': 2.0}
    return mapping.get(value, 0.0)

def convert_OrgSize(value):
    try:
        a, b = map(int, value.split('-'))
        return np.log(a)
    except (ValueError, TypeError):
          return 0.0


def parse_stackoverflow_data():

  df = pd.read_csv('eggs.csv.gz', dtype=str,compression='gzip', keep_default_na=False)
  # Apply the conversion functions to the DataFrame columns
  df['CareerSat'] = df['CareerSat'].apply(convert_CareerSat)
  df['MgrWant'] = df['MgrWant'].apply(convert_boolean)
  df['Age'] = df['Age'].apply(convert_integer)
  df['CodeRevHrs'] = df['CodeRevHrs'].apply(convert_integer)
  df['ConvertedComp'] = df['ConvertedComp'].apply(convert_integer)
  df['Dependents'] = df['Dependents'].apply(convert_boolean)
  df['DevEnvironVSC'] = df['DevEnvironVSC'].apply(convert_boolean)
  df['DevTypeFullStack'] = df['DevTypeFullStack'].apply(convert_boolean)
  df['EdLevel'] = df['EdLevel'].apply(convert_EdLevel)
  df['EduOtherMOOC'] = df['EduOtherMOOC'].apply(convert_boolean)
  df['EduOtherSelf'] = df['EduOtherSelf'].apply(convert_boolean)
  df['Extraversion'] = df['Extraversion'].apply(convert_boolean)
  df['GenderIsMan'] = df['GenderIsMan'].apply(convert_boolean)
  df['Hobbyist'] = df['Hobbyist'].apply(convert_boolean)
  df['MgrIdiot'] = df['MgrIdiot'].apply(convert_MgrIdiot)
  df['OpSys'] = df['OpSys'].apply(convert_OpSys)
  df['OpenSourcer'] = df['OpenSourcer'].apply(convert_OpenSourcer)
  df['OrgSize'] = df['OrgSize'].apply(convert_OrgSize)
  df['Student'] = df['Student'].apply(convert_boolean)
  df['UndergradMajorIsComputerScience'] = df['UndergradMajorIsComputerScience'].apply(convert_boolean)
  df['UnitTestsProcess'] = df['UnitTestsProcess'].apply(convert_boolean)
  df['WorkWeekHrs'] = df['WorkWeekHrs'].apply(convert_integer)
  df['YearsCode'] = df['YearsCode'].apply(convert_integer)
  df['YearsCodePro'] = df['YearsCodePro'].apply(convert_integer)

  # Columns to ignore
  columns_to_ignore = ['Country', 'EduOtherSElf','Respondent']
  df = df.drop(columns=columns_to_ignore,errors='ignore')
  return df

In [ ]:
# df
df = parse_stackoverflow_data()
df

,CareerSat,MgrWant,Hobbyist,OpenSourcer,Student,EdLevel,UndergradMajorIsComputerScience,EduOtherMOOC,EduOtherSelf,OrgSize,...,ConvertedComp,WorkWeekHrs,CodeRevHrs,UnitTestsProcess,DevEnvironVSC,OpSys,Extraversion,Age,GenderIsMan,Dependents
0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,4.605170,...,61000.0,80.00,0.0,0.0,1.0,-1.0,1.0,22.0,1.0,0.0
1,-2.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,0.000000,...,0.0,55.00,0.0,1.0,0.0,1.0,1.0,30.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,...,0.0,32.00,6.0,0.0,0.0,-1.0,0.0,42.0,1.0,1.0
3,1.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,2.302585,...,95179.0,32.00,12.0,0.0,0.0,0.0,1.0,23.0,1.0,0.0
4,-1.0,1.0,1.0,2.0,0.0,1.5,0.0,0.0,0.0,0.000000,...,13293.0,70.00,4.0,1.0,1.0,-1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65674,2.0,1.0,1.0,0.5,0.0,1.0,1.0,1.0,1.0,2.995732,...,130000.0,40.00,3.0,0.0,1.0,0.0,0.0,26.0,1.0,0.0
65675,1.0,1.0,0.0,0.0,0.0,1.5,1.0,0.0,0.0,2.995732,...,82488.0,37.75,10.0,1.0,0.0,-1.0,0.0,34.0,1.0,0.0
65676,0.0,0.0,0.0,2.0,0.0,1.5,1.0,1.0,1.0,4.605170,...,68745.0,39.00,10.0,1.0,1.0,0.0,0.0,37.0,1.0,0.0
65677,-1.0,1.0,1.0,0.0,1.0,1.5,1.0,0.0,0.0,4.605170,...,588012.0,40.00,5.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0


### Q2 Splitting Data

Now we prepare the converted data for regression. In this step, we:

 1. Extract the data as a numpy array
 2. Split the data into train and validation sets.  You can use the first 20% of the data (rounded down) as the validation set and keep the remaining as the training set. (Note that it is common practice to randomize the dataset; this has already been done. Don't shuffle the dataset for this assignment.)
 3. Split each set into the predicted column (the first column in the data frame) and the feature columns (the remaining columns), plus a final column corresponding to a constant 1.0 value.  Not that you should keep the ordering of the feature columns the same as they appear in the data.


In [ ]:
import numpy as np

def split_data(df):
    """Split the data into training and validation sets, and convert them to np.ndarray.

    Args:
        df : pandas.DataFrame -- the parsed data, as returned by parse_stackoverflow_data()

    Returns:
        X_train : np.ndarray -- the second 80% of the data features
        y_train : np.ndarray -- the second 80% of the target values
        X_val : np.ndarray -- the first 20% (rounded down) of the data features
        y_val : np.ndarray -- the first 20% of the target values
    """

    # Function to add a constant column
    def add_constant_column(X):
        return np.c_[X, np.ones(X.shape[0])]

    # Select relevant columns for regression
    regression_columns = [
        'CareerSat', 'MgrWant', 'Age', 'CodeRevHrs', 'ConvertedComp', 'Dependents',
        'DevEnvironVSC', 'DevTypeFullStack', 'EdLevel', 'EduOtherMOOC', 'EduOtherSelf',
        'Extraversion', 'GenderIsMan', 'Hobbyist', 'MgrIdiot', 'OpSys', 'OpenSourcer',
        'OrgSize', 'Student', 'UndergradMajorIsComputerScience', 'UnitTestsProcess',
        'WorkWeekHrs', 'YearsCode', 'YearsCodePro'
    ]

    # Extract the selected columns as a NumPy array
    X = df[regression_columns].values

    # Calculate the index to split the data (first 20% as the validation set)
    split_index = int(0.2 * len(X))

    # Split the data
    X_val, X_train = X[:split_index], X[split_index:]
    y_val, y_train = X_val[:, 0], X_train[:, 0]  # Assuming the target variable is the first column (CareerSat)

    # Extracting the feature columns and adding a constant column
    X_train_features = X_train[:, 1:]
    X_val_features = X_val[:, 1:]

    X_train_with_constant = add_constant_column(X_train_features)
    X_val_with_constant = add_constant_column(X_val_features)

    return X_train_with_constant, y_train, X_val_with_constant, y_val

# Example usage:
X_train, y_train, X_val, y_val = split_data(df)

# Print the shapes of the resulting arrays
print("Training Set Shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("\nValidation Set Shapes:")
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


Training Set Shapes:
X_train shape: (52544, 24)
y_train shape: (52544,)

Validation Set Shapes:
X_val shape: (13135, 24)
y_val shape: (13135,)


### Q3 Linear Regression

Now we are going to build a simple scikit-learn-like class for least squares linear regression.  Recall from lecture that the linear regression approach models the data as:
$$ y^{(i)} \approx \theta^T x^{(i)} $$
and the optimal $\theta$ is given by:
$$ \theta^* = (X^T X)^{-1}X^T y $$
using the notation described in the slides and course notes.  Recall, as mentioned in class, that you should use the `np.linalg.solve()` function rather than the `np.linalg.inv()` function to compute this solution.

Implement the class below, plus the `squared_error` function.

In [ ]:
# Assuming you have already loaded and converted the data into a DataFrame named df
# Assuming X_train, y_train, X_val, y_val are already defined from the previous steps

class LinearRegression:
    def __init__(self, X, y):
        X_with_constant = np.c_[np.ones(X.shape[0]), X]
        self.theta = np.linalg.solve(X_with_constant.T @ X_with_constant, X_with_constant.T @ y)

    def predict(self, X):
        X_with_constant = np.c_[np.ones(X.shape[0]), X]
        return X_with_constant @ self.theta

def squared_error(y_pred, y):
    return np.mean((y_pred - y) ** 2)

# Use the provided LinearRegression class
model = LinearRegression(X_train, y_train)

# Predict target values for the validation set
y_val_pred = model.predict(X_val)

# Calculate squared error for the validation set
error = squared_error(y_val_pred, y_val)

# Print the learned parameters and squared error
print("Learned Parameters (theta):", model.theta)
print("Squared Error on Validation Set:", error)

Learned Parameters (theta): [ 3.26290725e+14 -1.22753039e-02 -1.98830876e-02 -9.24801146e-03
  1.62795646e-07  1.86697977e-01 -3.76867153e-01  4.75457285e-02
 -4.21768099e-01  1.71048815e+15 -1.71048815e+15 -2.95974750e-01
 -1.20863196e+00 -2.82849895e-01  2.78251401e-01  6.34551205e-02
 -6.80780398e-02 -8.81631114e-02 -6.90007908e-01 -6.32044522e-01
 -7.74876034e-05 -1.79681052e-03 -4.78861978e-02  3.10997778e-02
 -3.26290725e+14]
Squared Error on Validation Set: 2.1353858369813477


## Q4 Evaluation versus baselines

As a final consideration, If you implement this properly, you will see that we get a squared error of around 1.3 on the validation set.  Is this "good"?  This is one of the more subtle points of many data science problems, but we can start to get some sense of this by looking at what the predictions would look like if we _just_ predicted the mean target output on the training set (this is essentially the "simplest" prediction we could make if we didn't look at the features at all).

Implement the following function to evaluate our linear regression.  Think about what this signifies about the quality of the solution.

In [ ]:
# @mugrade.local_tests
# @mugrade.local_tests
def evaluate_linear_regression(X_train, y_train, X_val, y_val):
    """ Evaluate the squared error of linear regression versus the simple mean-prediction baseline.

    Args:
        X_train : np.ndarray -- training set features
        y_train : np.ndarray -- training set target values
        X_val : np.ndarray -- validation set features
        y_val : np.ndarray -- validation set target values

    Returns:
        Tuple[validation_mse, baseline_mse]:
            validation_mse : float -- squared error of predictions on the validation set when training on the training set
            baseline_mse : float -- squared error of predicting the mean on the training set
    """
    # Train linear regression model on the training set
    model = LinearRegression(X_train, y_train)

    # Evaluate linear regression on the validation set
    y_val_pred = model.predict(X_val)
    validation_mse = squared_error(y_val_pred, y_val)

    # Calculate baseline (mean prediction) squared error on the training set
    baseline_mse = squared_error(np.mean(y_train) * np.ones_like(y_val), y_val)

    return validation_mse, baseline_mse

In [ ]:
validation_mse, baseline_mse = evaluate_linear_regression(X_train, y_train, X_val, y_val)

# Print the results
print("Validation MSE of Linear Regression:", validation_mse)
print("Baseline MSE (Mean Prediction):", baseline_mse)


Validation MSE of Linear Regression: 2.1353858369813477
Baseline MSE (Mean Prediction): 1.3525080981124082
